In [1]:
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report

import nibabel as nib

import torch
import torch.optim as optim
from torch.optim.lr_scheduler import MultiplicativeLR

from torchsummary import summary

%matplotlib inline

They have only one size.

In [2]:
# set(nib.load(b).get_fdata().shape for b in glob('blurred/*.BRIK'))

## Model

In [3]:
from model import *

In [4]:
model = DeepBrain(inplanes=179, planes=3)

G:\My Drive\termpj\model.py:102: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(m.weight, gain=nn.init.calculate_gain('relu'))


In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(summary(model, input_size=(179, 72, 72, 36)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1        [-1, 3, 72, 72, 36]             540
       BatchNorm3d-2        [-1, 3, 72, 72, 36]               6
              ReLU-3        [-1, 3, 72, 72, 36]               0
            Conv3d-4       [-1, 16, 36, 36, 18]           1,312
       BatchNorm3d-5       [-1, 16, 36, 36, 18]              32
              ReLU-6       [-1, 16, 36, 36, 18]               0
            Conv3d-7       [-1, 32, 36, 36, 18]             544
       BatchNorm3d-8       [-1, 32, 36, 36, 18]              64
            Conv3d-9       [-1, 32, 36, 36, 18]          13,856
      BatchNorm3d-10       [-1, 32, 36, 36, 18]              64
             ReLU-11       [-1, 32, 36, 36, 18]               0
           Conv3d-12       [-1, 32, 36, 36, 18]          27,680
      BatchNorm3d-13       [-1, 32, 36, 36, 18]              64
             ReLU-14       [-1, 32, 36,

C:\Users\pha\anaconda3\lib\site-packages\torch\nn\modules\container.py:100: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


## DataLoader

In [6]:
from torch.utils.data import DataLoader, Dataset

class MyDataset(Dataset):
    
    def __init__(self, datatype='blur', test=False):
        
        if datatype == 'blur':
            self.datafiles = glob('blurred/*.BRIK')
            
        elif datatype == 'scale':
            self.datafiles = glob('scaled/*.BRIK')
            
        else:
            pass
    
        if test: self.datafiles[-5:]
        else: self.datafiles[:-5]
        self.demographics = pd.read_csv('participants_with_runs.csv')
        
    def __getitem__(self, idx):
        
        x = torch.tensor(nib.load(self.datafiles[idx]).get_fdata()).float()
        y = torch.tensor(self.demographics.loc[idx].YB).float()
        
        return x, y
    
    def __len__(self):
        return len(self.datafiles)

In [7]:
dset = MyDataset('scale')
dloader = DataLoader(dset)

## Run

In [8]:
# loss_fn = nn.BCELoss(weight=torch.Tensor([1/44, 1/25]))
weight = torch.FloatTensor([(1/44)**2, (1/25)**2]).to(device)
# loss_fn = nn.CrossEntropyLoss(weight=torch.FloatTensor([1/44, 1/25]).to(device))
loss_fn = nn.CrossEntropyLoss(weight=weight)
# loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
scheduler = MultiplicativeLR(optimizer, lr_lambda=lambda epoch: 0.97)

In [9]:
def to_one_hot(y):
    
    return torch.tensor([[0, 1]]) if y else torch.tensor([[1, 0]])

def to_np(t):
    
    return t.cpu().detach().numpy()

In [10]:
# for i, (x, y) in enumerate(dloader):
    
#     if i == 0:
#         print(x, to_one_hot(y))
        
#     else:
#         break

In [ ]:
losses, ans, accs = [], [], []
for e in range(100):
    
    print(f"Epoch {e}")
    epoch_pred, epoch_true = [], []
    bth_loss = 0
    for i, (x, y) in enumerate(dloader):
        
        x, y = x.permute(0, 4, 1, 2, 3).to(device), y.to(device)
        epoch_true.append(round(float(y)))
        
        optimizer.zero_grad()
        
        y_pred = model.forward(x).to(device)
        epoch_pred.append(to_np(y_pred.argmax()))
        
        loss = loss_fn(y_pred, y.type(torch.long))
        bth_loss += loss.item()
        
        loss.backward()
        optimizer.step()
#         scheduler.step()
        
    torch.cuda.empty_cache()
    
    losses.append(bth_loss / len(dloader))
    ans.append((epoch_true, epoch_pred))
    
#     if e % 10 == 0:
#         plt.title(f'Loss of Epoch {e+1}')
#         plt.plot(losses)
#         plt.grid()
#         plt.close()
        
#         plt.title(f'Accuracy of Epoch {e+1}')
#         plt.plot(accs)
#         plt.grid()
#         plt.close()
    
    print(classification_report(epoch_true, np.array(epoch_pred) > 0.5))
    accs.append(classification_report(epoch_true, np.array(epoch_pred) > 0.5).split('\n')[5].split()[1])

Epoch 0
              precision    recall  f1-score   support

           0       0.63      0.86      0.73        44
           1       0.33      0.12      0.18        25

    accuracy                           0.59        69
   macro avg       0.48      0.49      0.45        69
weighted avg       0.52      0.59      0.53        69

Epoch 1
              precision    recall  f1-score   support

           0       0.61      0.75      0.67        44
           1       0.27      0.16      0.20        25

    accuracy                           0.54        69
   macro avg       0.44      0.46      0.44        69
weighted avg       0.49      0.54      0.50        69

Epoch 2
              precision    recall  f1-score   support

           0       0.63      0.95      0.76        44
           1       0.00      0.00      0.00        25

    accuracy                           0.61        69
   macro avg       0.31      0.48      0.38        69
weighted avg       0.40      0.61      0.48       

C:\Users\pha\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.64      1.00      0.78        44
           1       0.00      0.00      0.00        25

    accuracy                           0.64        69
   macro avg       0.32      0.50      0.39        69
weighted avg       0.41      0.64      0.50        69

Epoch 10
              precision    recall  f1-score   support

           0       0.64      1.00      0.78        44
           1       0.00      0.00      0.00        25

    accuracy                           0.64        69
   macro avg       0.32      0.50      0.39        69
weighted avg       0.41      0.64      0.50        69

Epoch 11
              precision    recall  f1-score   support

           0       0.64      1.00      0.78        44
           1       0.00      0.00      0.00        25

    accuracy                           0.64        69
   macro avg       0.32      0.50      0.39        69
weighted avg       0.41      0.64      0.50        69

E

In [ ]:
plt.title(f'Loss of Epoch {e+1}')
plt.plot(losses)
plt.grid()

In [ ]:
plt.title(f'Accuracy of Epoch {e+1}')
plt.plot(accs)
plt.grid()

## Guided Backpropagation

In [ ]:
# torch.save(model, 'model.pth')
model= torch.load('model_scale.pth')

In [ ]:
import nibabel as nib
from guided_backprop import GuidedBackprop

_ = model.eval()

# Remove LogSoftmax
model.classifier = nn.Sequential(*list(model.classifier.children())[:-1])

GBP = GuidedBackprop(model)

In [ ]:
x, y = dset[1]

In [ ]:
x.permute(3, 0, 1, 2).shape

In [ ]:
inputs, label = x.permute(3, 0, 1, 2), torch.BoolTensor(0)
input_img = nn.Parameter(torch.FloatTensor(inputs).unsqueeze(0), requires_grad=True).to(device)

guided_grads = GBP.generate_gradients(input_img, label)

export_gradient = np.zeros((3,75,93,81))
# export_gradient[:, 8:-8, 8:-8, :-10] = guided_grads
# nifti_img = nib.Nifti1Image(export_gradient.transpose(1, 2, 3, 0), np.eye(4))
nifti_img = nib.Nifti1Image(guided_grads.transpose(1, 2, 3, 0), np.eye(4))
nifti_img.to_filename('vis_tmp.nii.gz')

In [ ]:
nib.load('vis_tmp.nii.gz').get_data()[[nib.load('vis_tmp.nii.gz').get_fdata() != 0]]